# Read me

# Inputs

In [ ]:
import pandas as pd
import math
import numpy as np

In [ ]:
#Parameters
#Length of the interval on which the signature is computed
length_a=3
#Depth of the signature (order between 2 and 4)
sig_order=3
#Load a file with a time column and a Price column
data_csv=pd.read_csv(r"path//")
dataset=data_csv

In [ ]:
#Build returns 
dataset['Log_return']=np.log(dataset['Price'])-np.log(BTC_price_time['Price'].shift(+1))
dataset['Sign_return']=np.where(dataset.Log_return>=0,1,0)


# Please run the code here below

In [ ]:
from datetime import datetime
from matplotlib import pyplot as plt
import esig.tosig as ts
import sklearn
import xgboost
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def GetWindow(x,h_window =30,f_window=10):

    # First window
    X = np.array(x.iloc[:h_window,]).reshape(1,-1)
   
    # Append next window
    for i in range(1,len(x)-h_window+1):
        x_i = np.array(x.iloc[i:i+h_window,]).reshape(1,-1)
        X = np.append(X,x_i, axis=0)
        
    # Cut the end that we can't use to predict future price
    rolling_window = (pd.DataFrame(X)).iloc[:-f_window,]
    return rolling_window

#input = panda, historical window, future window
def GetNextMean(x,h_window=30,f_window=10):
    return pd.DataFrame((x.rolling(f_window).mean().iloc[h_window+f_window-1:,]))

#Function add time to the data set
def AddTime(X):
    t = np.linspace(0,1,len(X))
    return np.c_[t, X]


#Function for Lead lag transform
def Lead(X):
    
    s = X.shape
    x_0 = X[:,0]
    Lead = np.delete(np.repeat(x_0,2),0).reshape(-1,1)
     
    for j in range(1,s[1]):
        x_j = X[:,j]
        x_j_lead = np.delete(np.repeat(x_j,2),0).reshape(-1,1)
        Lead = np.concatenate((Lead,x_j_lead), axis =1)
     
    return Lead

#Function for Lead lag transform
def Lag(X):
    
    s = X.shape
    x_0 = X[:,0]
    Lag = np.delete(np.repeat(x_0,2),-1).reshape(-1,1)
  
    for j in range(1,s[1]):
        x_j = X[:,j]
        x_j_lag  = np.delete(np.repeat(x_j,2),-1).reshape(-1,1)
        Lag = np.concatenate((Lag,x_j_lag), axis = 1)
        
    return Lag
        

# We use only open price
close_price = dataset.loc[:,'Log_return']
h_window = length_a
f_window = 1
sig_level = sig_order

# mean next price
y = GetNextMean(close_price, h_window = h_window , f_window = f_window)

#GetNextMean(close_price, h_window = h_window , f_window = f_window)

# normal window features
X_window = AddTime(GetWindow(close_price, h_window = h_window, f_window = f_window))
X_window = pd.DataFrame(X_window)


# signature features
#Consider only area that has at least f_window future prices left
close_price_slice = close_price.iloc[0:(len(close_price)-(f_window))]
close_price_array = np.array(close_price_slice).reshape(-1,1)
lag = Lag(close_price_array)
lead = Lead(AddTime(close_price_array))
#concatenate everything to get a datastream
stream = np.concatenate((lead,lag), axis = 1)
X_sig = [ts.stream2sig(stream[0:2*h_window-1], sig_level)]
print(stream[11])
for i in range(1,(len(close_price)-(f_window)-(h_window)+1)):
    stream_i = stream[2*i: 2*(i+h_window)-1]
    signature_i = [ts.stream2sig(stream_i, sig_level)]
    X_sig = np.append(X_sig, signature_i, axis=0)

X_sig = pd.DataFrame(X_sig)

from sklearn import decomposition as decomp
from sklearn.preprocessing import StandardScaler


scaler=StandardScaler()#instantiate
scaler.fit(X_sig) # compute the mean and standard which will be used in the next command
X_scaled=scaler.transform(X_sig)



# Paradigm 1

In [ ]:
y=BTC_price_time.loc[:,'Sign_return']
#provide the wished length of the test set
test_len = 2000
train_len = len(y) - test_len
X_train = X_scaled[:train_len-h_window,]
y_train = y.iloc[h_window:train_len,]
X_test = X_scaled[train_len-h_window:,]
y_test = y.iloc[train_len:len(y),]
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)
X_train = X_train.fillna(X_train.mean())
y_train= y_train.fillna(y_train.mean())
X_test = X_test.fillna(X_train.mean())
y_test= y_test.fillna(y_train.mean())


In [ ]:
#Results with Elastic Net
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error
# sklearn.linear_model.SGDClassifier(loss='log', penalty='elasticnet', alpha=0.0001, l1_ratio=0.15)
param_search = {'alpha': [1e-5,1e-4,1e-3, 1e-2,1e-1,2e-1, 5e-1, 1], 'l1_ratio':[0.15,0.3,0.45,0.6,0.75,0.9]}
myModel = GridSearchCV(estimator=SGDClassifier(alpha = 0.1,
                                       
                                    
                                       max_iter = 10e5,
                                       penalty='elasticnet',
                                       loss='log'),
                        param_grid = param_search,
                        cv = TimeSeriesSplit(n_splits=8),
                        n_jobs=-1)

myModel.fit(X_train, y_train)

#Make predictions
y_train_predict = myModel.predict(X_train)
y_test_predict = myModel.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_predict)
print("confusion matrix with an multiple logistic regression model with Elastic Net regularisation")
print(cm)


In [ ]:
#Results with Random Forest

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

#mean absolute percentage error
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#Perform a GridsearchCV
param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : [ 'l2'],
    'classifier__C' : np.logspace(-20, -4, 4, 20, 50),
    'classifier__solver' : ['newton-cg']},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,150,160)),
    'classifier__max_features' : list(range(5,20, 40))}
]
pipe = Pipeline([('classifier' , RandomForestClassifier())])
myModel = GridSearchCV(pipe, param_grid = param_grid, cv = 8, verbose=True, n_jobs=-3)

# myModel.fit(X_scaled[:len(X_scaled)-240-1], values[h_window+1:aa-240])
myModel.fit(X_train, y_train)
#Make predictions
# y_train_predict = myModel.predict(X_scaled)
y_test_predict = myModel.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_predict)
print("confusion matrix with a RF model")
print(cm)

In [ ]:
#Results with XGBoost

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV  #Perforing grid search

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

#GridsearchCV    
param_search = {
    'max_depth':[4,5,6]
    ,'min_child_weight':[4,5,6]
    ,'gamma':[i/10.0 for i in range(0,5)]
}

myModel = GridSearchCV(estimator=XGBRegressor(
                        objective='binary:logistic',
                        learning_rate=0.01,
                        n_estimators=500,
                        max_depth=5,
                        min_child_weight=5,
                        gamma=0,
                        subsample=0.8,
                        colsample_bytree=0.8, 
                        eval_metric ='mae',
                        reg_alpha=0.05
                        ),
                       param_grid = param_search,
                       cv = TimeSeriesSplit(n_splits=4),n_jobs=-1
                      )

#Fit model
myModel.fit(X_train, y_train)

#Make predictions
# y_train_predict = myModel.predict(X_train)
y_test_predict = myModel.predict(X_test)


positif=0
negatif=0
extreme=0
inter=0
i=0
hg,hd,bg,bd=0
while i < len(y_test_predict):
    if y_test_predict[i]>0.5:
        if np.asscalar(y_test.iloc[i])>0.5:
            hg+=1
            extreme+=1
        else:
            hd+=1
            extreme+=1
    elif y_test_predict[i]<0.5:
        if np.asscalar(y_test.iloc[i])<0.5:
            bd+=1
        else:
            bg+=1
    else:
        inter+=1
    i+=1
    
print("confusion matrix with an XGB model")
print('[',hg,hd)
print(' ', bg,bd,']')

# Paradigm 2

In [ ]:
#forecast horizon
h=300

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from pmdarima import auto_arima

#Forecast signature terms with ARIMA
o=2
list_col=[]
X_scaled=pd.DataFrame(X_scaled)
X_scaled=X_scaled.fillna(X_scaled.mean())
while o<3:
    series=X_scaled.iloc[len(X_scaled)-1000:,o]
    print(len(series))
    p=0
    list_rows=[]
    param=h
    while p<param:
        train = series[:len(series)-param+p]
        test = series[len(series)-param+p:len(series)-param+p+1]
        if p==0:
            print('test',test)
        model= auto_arima(series, start_p=0, start_q=0,
                               max_p=3, max_q=3, 
                               seasonal=False,
                               d=1, trace=False,
                               error_action='ignore',  
                               suppress_warnings=True, 
                               stepwise=True)

        
        model.fit(train)
        future_forecast = model.predict(n_periods=len(test))
        list_rows.append(future_forecast)
        
        p+=1
    list_col.append(list_rows)
    print(list_col)
    
    o+=1

In [ ]:
true_returns=dataset['Log_return'][len(dataset['Log_return'])-h:len(dataset['Log_return'])]
to_add=dataset['Log_return'][len(dataset['Log_return'])-h-length_a:len(dataset['Log_return'])-length_a]
list_returns_for=[]
k=0
for i in list_col:
    list_returns_for.append(i+np.asarray(to_add[k:k+1,]))
    k+=1
    
good=0
bad=0
m=0
while m<h:
    if (i[m].item()>0 and true_returns[m:m+1,].item()>0)or (i[m].item()<0 and true_returns[m:m+1,].item()<0):
        good+=1
    m+=1
print("accuracy of the paradigm 2")
print(good/h)

# Paradigm 3

In [ ]:
import random
y_vector=y.iloc[h_window:len(y)]
y_train=y_vector[9:]
list_x_train=[]
k=9
while k<len(X_scaled):
    list_temp=[]
    p=0
    while p<=9:
        list_temp.append(X_scaled[k-p])
        p+=1
    list_x_train.append(list_temp)
    k+=1

list_fin_train=[]
for i in list_x_train:
    list_temp=[]
    for j in i:
        for k in j:
            list_temp.append(k)
    list_fin_train.append(list_temp)

X_full=np.asarray(list_fin_train)
y_full=np.asarray(y_train).reshape((len(y_train), 1))

X_train=X_full[1:len(X_full)-1999]
X_test=X_full[len(X_full)-1999:]

y_train=y_full[1:len(y_full)-1999]
y_test=y_full[len(y_full)-1999:]



In [ ]:
def sigmoid(x):
    return 1.0/(1+ np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)
def ReLU(x): 
    j=0
    for i in x:
        p=0
        for k in i:
            if k <0:
                x[j][p]=0
            p+=1
        j+=1
    return x
class NeuralNetwork:
    def __init__(self, x, y, lamb1, learning_r):
        self.input = x
        list_weights1 = []
        j=0
        while j<self.input.shape[1]:
            list_temp=[]
            k=j//40
            m=0
            list_temp+=[0]*180
            r=18*(k+1)
            b=r-18
            for i in range(b,r):
                list_temp[i]=random.uniform(-150,150)
            list_weights1.append(np.asarray(list_temp))
            j+=1
            
        self.weights1   = np.asarray(list_weights1)
        
        list_weights2=[]
        j=0
        while j<180:
            list_temp=[]
            k=j//18
            m=0
            list_temp+=[0]*60
            r=6*(k+1)
            b=r-6
            for i in range(b,r):
                list_temp[i]=random.uniform(-150,150)
            list_weights2.append(np.asarray(list_temp))
            j+=1
        
        self.weights2   = np.asarray(list_weights2)
        
        list_weights3=[]
        j=0
        while j<60:
            list_temp=[]
            k=j//6
            m=0
            list_temp+=[0]*10
            r=1*(k+1)
            b=r-1
            for i in range(b,r):
                list_temp[i]=random.uniform(-15,15)
            list_weights3.append(np.asarray(list_temp))
            j+=1
        self.weights3   = np.asarray(list_weights3)
        self.weights4   = np.random.uniform(low=-15,high=15, size=(10,1))
        self.weights5   = np.random.uniform(low=-15,high=15, size=(1,1))
        self.y          = y
        self.output     = np.zeros(self.y.shape)
        
        self.lamb1=lamb1
        self.learning_r=learning_r
        
    def feedforward(self):
        #ON peut tenter avec ReLU aussi 
#         self.layer1=ReLU(np.dot(self.input, self.weights1))
#         self.layer2=ReLU(np.dot(self.layer1, self.weights2))
#         self.layer3=ReLU(np.dot(self.layer2, self.weights3))
#         self.layer4=ReLU(np.dot(self.layer3, self.weights4))
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.layer2 = sigmoid(np.dot(self.layer1, self.weights2))
        self.layer3 = sigmoid(np.dot(self.layer2, self.weights3))
        self.layer4 = sigmoid(np.dot(self.layer3, self.weights4))
        self.output = sigmoid(np.dot(self.layer4, self.weights5))
        
    def backprop(self):
        # application of the chain rule to find derivative of the loss function with respect to weights
        self.list3=[]
        self.list2=[]
        self.list1=[]
        self.list_input=[]
        self.list_weights3=[]
        self.list_weights2=[]
        p=0
        while p<10:
            list3=[]
            
            for i in self.layer3:
                list3.append(i[p])
            list2=[]
            for i in self.layer2:
                list2.append(i[p*6:(p+1)*6])
            list1=[]
            for i in self.layer1:
                list1.append(i[p*18:(p+1)*18])
            list_input=[]
            for i in self.input:
                list_input.append(i[p*40:(p+1)*40])
            list_weights3=[]
            
            f=0
            for i in self.weights3:
                if f>=(p)*6 and f<(p+1)*6:
                    list_weights3.append(i[p])
                f+=1

            list_weights2=[]

            for i in self.weights2[p*18:(p+1)*18]:
                list_weights2.append(i[p*6:(p+1)*6])

            
            self.list_input.append(np.asarray(list_input))
            self.list1.append(np.asarray(list1))
            self.list2.append(np.asarray(list2))
            self.list3.append(np.asarray(list3))
            self.list_weights3.append(np.asarray(list_weights3))
            self.list_weights2.append(np.asarray(list_weights2))
            
            p+=1
        
        self.list_input=np.asarray(self.list_input)     
        self.list1=np.asarray(self.list1)
        self.list2=np.asarray(self.list2)
        self.list3=np.asarray(self.list3)
        self.list3=self.list3.reshape(10,7988,1)
        self.list_weights3=np.asarray(self.list_weights3)
        self.list_weights2=np.asarray(self.list_weights2)
        
        lambda_1=self.lamb1
        lambda_2=0.5
        learning_rate=self.learning_r
        error_output=(self.y*np.log(self.output)+(1-self.y)*np.log(1-self.output))
        d_error_output=self.y/self.output +(1-self.y)/(self.output-1)

        d_weights5 = np.dot(self.layer4.T, (-2*(self.y-self.output) * sigmoid_derivative(self.output)))
        d_weights4 = np.dot(self.layer3.T, (np.dot((-2*(self.y-self.output) * sigmoid_derivative(self.output)), self.weights5.T) * sigmoid_derivative(self.layer4)))
        error_layer3=(np.dot((np.dot(-2*(self.y-self.output) * sigmoid_derivative(self.output), self.weights5.T) * sigmoid_derivative(self.layer4)),self.weights4.T))
        error_layer2=np.dot((np.dot((np.dot(-2*(self.y-self.output) * sigmoid_derivative(self.output), self.weights5.T) * sigmoid_derivative(self.layer4)),self.weights4.T) * sigmoid_derivative(self.layer3)),self.weights3.T)
        error_layer1=np.dot((np.dot((np.dot((np.dot(-2*(self.y-self.output) * sigmoid_derivative(self.output), self.weights5.T) * sigmoid_derivative(self.layer4)),self.weights4.T) * sigmoid_derivative(self.layer3)),self.weights3.T)*sigmoid_derivative(self.layer2)),self.weights2.T)

        
        x=0
        error_neuron=[]
        while x<10:
            list_temp=[]
            for i in error_layer3:
                list_temp.append(i[x])
            error_neuron.append(np.asarray(list_temp))
            x+=1
            
        error_neuron=np.asarray(error_neuron) 
        f=0
        d_weights3=[]
        d_weights2=[]
        d_weights1=[]
        while f<10:
            d_weights3.append(np.dot(self.list2[f].T,(error_neuron[f].reshape(7988,1) * sigmoid_derivative(self.list3[f]))))
            d_weights2.append(np.dot(self.list1[f].T,(np.dot((error_neuron[f].reshape(7988,1) * sigmoid_derivative(self.list3[f])),self.list_weights3[f].reshape(6,1).T)*sigmoid_derivative(self.list2[f]))))
            d_weights1.append(np.dot(self.list_input[f].T,(np.dot((np.dot((error_neuron[f].reshape(7988,1) * sigmoid_derivative(self.list3[f])),self.list_weights3[f].reshape(6,1).T)*sigmoid_derivative(self.list2[f])),self.list_weights2[f].reshape(18,6).T)*sigmoid_derivative(self.list1[f]))))
            f+=1
        # update the weights with the derivative of the loss function

        self.weights5 -= learning_rate*d_weights5
        self.weights5 -= learning_rate*lambda_1*self.weights5

        self.weights4 -= learning_rate*d_weights4
        self.weights4 -= learning_rate*lambda_1*self.weights4
        s=0
        for d in d_weights3:
            for b in d:

                self.weights3[s][s//6]-=learning_rate*b
                s+=1
        
        
        
        e=0
        for i in d_weights2:
            for j in i:
                m=0
                for k in j:
                    self.weights2[e//6][(e//(18*6))*6+m]-=learning_rate*k
                    e+=1
                    m+=1
        
        e=0
        for i in d_weights1:
            for j in i:
                m=0
                for k in j:
                    self.weights1[e//18][(e//(40*18))*18+m]-=learning_rate*k
                    e+=1
                    m+=1
        
        self.weights3 -=learning_rate*lambda_1*self.weights3
        self.weights2 -=learning_rate*lambda_1*self.weights2
        self.weights1 -=learning_rate*lambda_1*self.weights1
        

    def forecast (self, X_test, Y_test):
        test_layer1 = sigmoid(np.dot(X_test, self.weights1))
        test_layer2 = sigmoid(np.dot(test_layer1, self.weights2))
        test_layer3 = sigmoid(np.dot(test_layer2, self.weights3))
        test_layer4 = sigmoid(np.dot(test_layer3, self.weights4))
        self.test_output = sigmoid(np.dot(test_layer4, self.weights5))
        k=0
        hg=0
        bd=0
        hd=0
        bg=0
        liste_matrix=[]
        for i in self.test_output:
            if (i>0.5 and Y_test[k]==1):
                bd +=1
            elif (i<0.5 and Y_test[k]==0):
                hg +=1
            elif(i<0.5 and Y_test[k]==1):
                bg+=1
            else:
                hd +=1
            k+=1
        liste_matrix.append(hg)
        liste_matrix.append(bg)
        liste_matrix.append(hd)
        liste_matrix.append(bd)
        return liste_matrix
        
if __name__ == "__main__":
#     nn = NeuralNetwork(X_train,y_train, 0.33, 0.09)
#     for i in range(100):
#         nn.feedforward()
#         nn.backprop()
#     test=nn.forecast(X_test, y_test)
#     print(nn.weights4)
#     print(nn.test_output)
        
    maxi_out=0
    liste_max=[]
    liste_matrice=[]
    list_w4=[]
    for p in np.arange(0.22,0.8,0.03):

        for j in np.arange(0,0.3,0.03):
            nn = NeuralNetwork(X_train,y_train, p, j)
            for i in range(60):
                nn.feedforward()
                nn.backprop()

            test=nn.forecast(X_test, y_test)
            if (test[0]+test[3])/(test[0]+test[1]+test[2]+test[3])>maxi_out:
                maxi_out=(test[0]+test[3])/(test[0]+test[1]+test[2]+test[3])
                liste_max.append((p,j))
                liste_matrice.append(test)
                list_w4.append(nn.weights4)
                print(test)

print("Accuracy of the best model is : " + str(maxi_out))
print(liste_max)
